# CASO DE NEGOCIO: Modelamiento de Retención de Clientes en Scholastic Travel Company

## Sinopsis

Scholastic Travel Company (STC) quiere usar los datos de sus clientes para predecir quién comprará un paquete de viaje el próximo año. Los casos presentan datos de 2.389 clientes, donde el conjunto A contiene varios campos de perfil y el conjunto B contiene información adicional del Net Promoted Score (NPS)

Este notebook ha sido preparado con información provista en el Caso de Negocios de Harvard UV7579 (Agosto 23, 2018), Retention Modeling at Scholastic Travel Company, elaborado por Anton Ovchinnikov, profesor de Management Science, Operations Management y Customer Analytics, en la escuela de negocios Smith, de Queen’s University en Canadá.

Consideraremos varios modelos de analítica de datos:

1. MÁQUINAS DE VECTORES DE SOPORTE: un método intermedio entre las regresiones y los árboles que se ajusta a planos de menor dimensión para separar los datos en clases con el máximo margen entre ellas



# Preparación de los datos

In [ ]:
# Cargar los paquetes y librerías necesarios 

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer

import warnings
warnings.filterwarnings("ignore")

# Cargar los datos desde el archivo CSV en el dataframe llamado df.
df = pd.read_csv("data_taller.csv", sep=";", header = 0)  

# Limpieza de datos -- parte 1: convertir tipos de datos incorrectos
df['From.Grade'] = df['From.Grade'].astype('category')
df['To.Grade'] = df['To.Grade'].astype('category')
df['Group.State'] = df['Group.State'].astype('category')
df['Is.Non.Annual.'] = df['Is.Non.Annual.'].astype('category')
df['Travel.Type'] = df['Travel.Type'].astype('category')
df['Poverty.Code'] = df['Poverty.Code'].astype('category')
df['CRM.Segment'] = df['CRM.Segment'].astype('category')
df['School.Type'] = df['School.Type'].astype('category')
df['Parent.Meeting.Flag'] = df['Parent.Meeting.Flag'].astype('category')
df['MDR.Low.Grade'] = df['MDR.Low.Grade'].astype('category')
df['MDR.High.Grade'] = df['MDR.High.Grade'].astype('category')
df['School.Sponsor'] = df['School.Sponsor'].astype('category')
df['SchoolGradeTypeLow'] = df['SchoolGradeTypeLow'].astype('category')
df['SchoolGradeTypeHigh'] = df['SchoolGradeTypeHigh'].astype('category')
df['GroupGradeTypeLow'] = df['GroupGradeTypeLow'].astype('category')
df['GroupGradeTypeHigh'] = df['GroupGradeTypeHigh'].astype('category')
df['MajorProgramCode'] = df['MajorProgramCode'].astype('category')
df['SingleGradeTripFlag'] = df['SingleGradeTripFlag'].astype('category')
df['SchoolSizeIndicator'] = df['SchoolSizeIndicator'].astype('category')
df['Retained.in.2012.'] = df['Retained.in.2012.'].astype('category')

df['Region'] = df['Region'].astype('category')
df['Special.Pay'] = df['Special.Pay'].astype('category')
df['Income.Level'] = df['Income.Level'].astype('category')
df['SPR.Product.Type'] = df['SPR.Product.Type'].astype('category')
df['SPR.New.Existing'] = df['SPR.New.Existing'].astype('category')

# Presentamos una función personalizada que llamaremos CombineRareCategories
# esta función tiene dos argumentos: el nombre del dataframe (data) y el número mínimo de puntos de datos para seguir 
# siendo una categoría separada (mincount)
# esta función recorrerá todas las columnas del marco de datos y combinará todas las categorías que aparezcan en los 
# datos menos que el número mínimo de veces en (Other)

def CombineRareCategories(data, mincount):
    for col in data.columns:
        if (type(data[col][0]) == str):
            for index, row in pd.DataFrame(data[col].value_counts()).iterrows():
                if ( row[0] < mincount):
                    df[col].replace(index, 'Other_' + col, inplace = True)
                else:
                    None

# Aplicamos esta función a variables con mincount=10                    
CombineRareCategories(df, 10)
# Reemplazo/Imputación de datos faltantes
pd.DataFrame(df).isna().sum() # Chequeamos si hay datos faltantes  

# Método:
# Variables Categóricas: agregar una nueva categoría 'missing_value' (como si fuera un nuevo color, o género)
# Variables Numéricas: reemplazar con la mediana (o la media, o el valor más frecuente, etc.) Un método alterno es
# ejecutar una imputación, see here: https://scikit-learn.org/stable/modules/impute.html 
# + agregamos columnas sustitutas indicando que el valor ha sido imputado

# creación de variables sustitutas
for col in df:
    if df[col].isna().sum() != 0: 
        df[col + '_surrogate'] = df[col].isna().astype(int)

# fijación de variables categóricas
imputer = SimpleImputer(missing_values = np.nan, strategy='constant')
imputer.fit(df.select_dtypes(exclude=['int64','float64']))
df[df.select_dtypes(exclude=['int64','float64']).columns] = imputer.transform(df.select_dtypes(exclude=['int64','float64']))
           
# fijación de variables numéricas 
imputer = SimpleImputer(missing_values = np.nan, strategy='median')
imputer.fit(df.select_dtypes(include=['int64','float64']))
df[df.select_dtypes(include=['int64','float64']).columns] = imputer.transform(df.select_dtypes(include=['int64','float64']))

# Creación de dummies para variables no numéricas ("one hot encoding")
df = pd.get_dummies(df, columns = df.select_dtypes(exclude=['int64','float64']).columns, drop_first = True)
pd.options.display.max_columns = None # remove the limit on the number of columns by default only 20 are shows

# Definición de X y Y
y = df['Retained.in.2012._1']
X = df.drop(columns = 'Retained.in.2012._1')
# Definimos la semilla para el generador de número aleatorios
np.random.seed(77300)

# Dividimos los datos aleatoriamente en 80% para entrenamiento y 20% para prueba 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, stratify=y)
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Modelo SVM

In [ ]:
# Desarrollamos un clasificador SVM LINEAL
clfl = svm.SVC(kernel='linear', C=1, probability=True, random_state=1234) # Linear Kernel

# Entrenamos el modelo usando el dataset de entrenamiento
clfl.fit(X_train, y_train)

# Importamos el módulo scikit-learn metrics para cálculo de Accuracy
from sklearn import metrics
#Pronosticamos la respuesta para el dataset de prueba
#En entrenamiento
y_pred_t = clfl.predict(X_train)
#En prueba
y_pred = clfl.predict(X_test)

#Accuracy del modelo:  Qué tan frecuentemente el clasificador es correcto?
#En entrenamiento: 
print("Accuracy SVM - entrenamiento: ",metrics.accuracy_score(y_train, y_pred_t))
#En prueba: 
print("Accuracy SVM - prueba: ",metrics.accuracy_score(y_test, y_pred))

Accuracy SVM - entrenamiento:  0.8273155416012559
Accuracy SVM - prueba:  0.805439330543933


# Modelo con Red Neuronal

In [ ]:
#librerías y paquetes
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import numpy as np 
import pandas as pd
import keras_tuner as tk # instalar por cmd
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import to_categorical
from matplotlib import pyplot

     |████████████████████████████████| 135 kB 4.9 MB/s 
     |████████████████████████████████| 1.6 MB 58.3 MB/s 


In [ ]:
# Función para probar hiperparámetros

def model_builder(hp):
    
  hp_activation = hp.Choice('activation', values=['relu', 'tanh','linear','sigmoid'])
  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=1000, step=50)
  hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=1000, step=50)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
 
  model = Sequential()
  model.add(tf.keras.layers.Dense(units=hp_layer_1, activation=hp_activation,input_dim=241))
  model.add(tf.keras.layers.Dense(units=hp_layer_2, activation=hp_activation))
  model.add(Dense(1, activation='sigmoid',use_bias=False))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='mean_squared_error',metrics=['accuracy'])
  
  return model


# Algoritmo Hyperband

tuner = tk.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=100,
                     factor=5,directory='/notebooks/Documents/NOTEBOOKS/RNCLASS-HP6',
                     project_name='RNCLASS-HP6')

# crieterio de parada anticipada
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# Búsqueda
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[stop_early],verbose=True)
best_model = tuner.get_best_models()[0]
#Resúmen
tuner.search_space_summary()
# Resúmen mejores parámetros
tuner.results_summary()



Trial 59 Complete [00h 00m 11s]
val_accuracy: 0.8117154836654663

Best val_accuracy So Far: 0.8158996105194092
Total elapsed time: 00h 04m 29s
Search space summary
Default search space size: 4
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'linear', 'sigmoid'], 'ordered': False}
layer_1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 50, 'sampling': None}
layer_2 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 50, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
Results summary
Results in /notebooks/Documents/NOTEBOOKS/RNCLASS-HP6/RNCLASS-HP6
Showing 10 best trials
Trial summary
Hyperparameters:
activation: linear
layer_1: 701
layer_2: 251
learning_rate: 0.0001
tuner/epochs: 100
tuner/initial_epoch: 20
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0052
Score: 0.8158996105194092
Trial summary
Hyper

In [ ]:
# Construcción del MODELO FINAL

"""
activation: linear
layer_1: 701
layer_2: 251
learning_rate: 0.0001
"""
model1 = Sequential()
   
model1.add(tf.keras.layers.Dense(units=701, activation="linear",input_dim=241))
model1.add(tf.keras.layers.Dense(units=251, activation="linear"))
model1.add(Dense(1, activation='sigmoid',use_bias=False))

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='mean_squared_error',metrics=['accuracy'])
model1.fit(X_train,y_train,epochs=200)
print(model1.summary())

#Pronosticamos la respuesta para el dataset de prueba
#En entrenamiento
y_pred_t = model1.predict(X_train)
#En prueba
y_pred = model1.predict(X_test)

class_threshold = 0.5
y_pred_t = np.where(y_pred_t > class_threshold, 1, 0) # clasificación
y_pred = np.where(y_pred > class_threshold, 1, 0) # clasificación


#Accuracy del modelo:  Qué tan frecuentemente el clasificador es correcto?
#En entrenamiento: 
print("Accuracy  NN- entrenamiento: ",metrics.accuracy_score(y_train, y_pred_t))
#En prueba: 
print("Accuracy NN - prueba: ",metrics.accuracy_score(y_test, y_pred))

Epoch 1/200
60/60 [==============================] - 1s 6ms/step - loss: 0.1883 - accuracy: 0.7111
Epoch 2/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1539 - accuracy: 0.7849
Epoch 3/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1462 - accuracy: 0.7975
Epoch 4/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1417 - accuracy: 0.8085
Epoch 5/200
60/60 [==============================] - 0s 7ms/step - loss: 0.1407 - accuracy: 0.8116
Epoch 6/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1355 - accuracy: 0.8179
Epoch 7/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1350 - accuracy: 0.8195
Epoch 8/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1337 - accuracy: 0.8168
Epoch 9/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1339 - accuracy: 0.8205
Epoch 10/200
60/60 [==============================] - 0s 6ms/step - loss: 0.1326 - accuracy: 0.8257
Epoch 11/

# Conclusiones

Como se puede observar tras utilizar el modelo con redes neuronales se presenta una mejora en el accuracy de entrenamiento llevandolo al 85,5% mientras que para el modelo de SVM se obtenia un 82,73%. Sin embargo, para el caso de prueba se observa un pequeño sobreajuste en el modelo de redes neuronales ya que el accuracy aquí disminuye al 79,49%, adicionalmente es menor al presentado por el modelo de SVM donde este es del 80,54%